In [1]:
using BenchmarkTools
macro atime(expr) :(@btime $expr samples=1 evals=1) end

@atime (macro with 1 method)

In [2]:
function get_η_(T)
    D = ndims(T)
    η = reverse(T)
    for d in 1:D
        cumsum!(η, η, dims=d)
    end
    reverse!(η)
end

get_η_ (generic function with 1 method)

In [3]:
function get_η_view!(T)
    D = ndims(T)
    η = view(T, reverse.(axes(T))...)
    for d in 1:D
        cumsum!(η, η, dims=d)
    end
    parent(η)
end

get_η_view(T) = get_η_view!(copy(T))

get_η_view (generic function with 1 method)

In [4]:
T = rand(1:8, 5, 4, 3)
A = get_η_(T)
B = get_η_view(T)
A == B

true

In [5]:
T = @atime rand(1:8, 400, 400, 400);

  450.770 ms (2 allocations: 488.28 MiB)


In [6]:
A = @atime get_η_($T);

  438.404 ms (10 allocations: 488.28 MiB)


In [7]:
B = @atime get_η_view($T);

  251.031 ms (14 allocations: 488.28 MiB)


In [8]:
A == B

true

In [9]:
tmp = @time copy(T)
C = @time get_η_view!(tmp)
A == C

  0.079798 seconds (16 allocations: 488.282 MiB, 1.88% gc time, 2.39% compilation time)
  0.178083 seconds (12 allocations: 608 bytes)


true